In [ ]:
pip install gql[all]

In [1]:
import requests
import json
import pandas as pd
import csv

Connecting to GraphQL

In [3]:
#Connecting to GraphQL
from gql import Client, gql
from gql.transport.requests import RequestsHTTPTransport

dgraph_url = "https://blue-surf-570083.us-east-1.aws.cloud.dgraph.io/graphql" # this is your GraphQL Dgraph endpoint

transport = RequestsHTTPTransport(
    url=dgraph_url, verify=True, retries=3,
)

client = Client(transport=transport, fetch_schema_from_transport=True)

In [4]:
#Reading files and creating multiple copies
df=pd.read_csv("Final_data_merged.csv")
df1= df
df2= df
df3=df


/Users/rajib/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5,6,347,390,489) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df3

,Unnamed: 0,index,word,category,cids,cids1,similar_word,pam_language,drl_language,gor_language,...,tkr_language,avi_language,hbn_language,akg_language,fud_language,ebr_language,alp_language,lml_language,hdy_language,mkd_language
0,0,1,I,Pronouns,9.2.3,NaN,NaN,ika/ikayu,nindo/ŋimba/ŋindu,yi7o/yiʔo,...,гъу/йигъын/шу,a/áɣɛ́,ŋa,au,akoe/ke/kee/koe/koutou,hɛ̀,ale,giḡo/xiNg~o,ate/ati/kiʔne,ти
1,1,2,thou,NaN,NaN,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
2,2,2,you,Pronouns,9.2.3,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
3,3,2,you,Prompters of attention,9.6.3.3,NaN,NaN,iya,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ittʼo/itʔo,тој
4,4,5,you,Pronouns,9.2.3,NaN,NaN,ila,-d̪iga,timoŋoliyo,...,мамы/манбы,ɲɔ̃́,nɛdiŋa/ŋindaŋá,duda,kiraua/lotou/lā/lāua/lătou,wò,sie,k|era,issuw/ittʔuw,тие
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,1187,207,name,Name of a thing,9.7.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1188,1188,207,title,"Title, name of honor",4.5.5.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1189,1189,207,title,"Land, property",6.5.1.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1190,1190,207,title,Name of a person,9.7.1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


ISO Language Loading

In [6]:
#Reading Column Names
df3.columns

Index(['Unnamed: 0', 'index', 'word', 'category', 'cids', 'cids1',
       'similar_word', 'pam_language', 'drl_language', 'gor_language',
       ...
       'tkr_language', 'avi_language', 'hbn_language', 'akg_language',
       'fud_language', 'ebr_language', 'alp_language', 'lml_language',
       'hdy_language', 'mkd_language'],
      dtype='object', length=784)

In [8]:
#Dropping Columns and storing it in a dataframe isomapping
isomapping=df3.drop(['Unnamed: 0','index','word','category','cids','cids1','similar_word','art_language'],axis=1)

In [9]:
#Reading Column Names and storing it as Label
label=isomapping.columns
label

Index(['pam_language', 'drl_language', 'gor_language', 'xbr_language',
       'nap_language', 'wew_language', 'uig_language', 'cmn_language',
       'guk_language', 'iba_language',
       ...
       'tkr_language', 'avi_language', 'hbn_language', 'akg_language',
       'fud_language', 'ebr_language', 'alp_language', 'lml_language',
       'hdy_language', 'mkd_language'],
      dtype='object', length=776)

In [11]:
#Separating the words in columns
ss=pd.DataFrame()
di={}

for i in label:
    d1=pd.DataFrame()
    call1=[]
    try:
        d1=(isomapping[i].str.split('/',expand=True))
        for z in range(len(d1.columns.to_list())):
            call1.append(str(i)+','+str(z))
        
        d1.columns=call1
        if len(ss)==0:
            ss=d1
        else:
            ss=pd.merge(d1,ss,how='inner',left_index=True,right_index=True)
    except:
        continue

In [18]:
#Extracting ISO639 Code from File
lab=ss.columns
da=[]
for i in lab:
    c=i.split('_')[0]
    if c not in da:
        da.append(c)

['mkd',
 'hdy',
 'lml',
 'alp',
 'ebr',
 'fud',
 'akg',
 'hbn',
 'avi',
 'tkr',
 'fij',
 'hop',
 'languages',
 'wbp',
 'cwg',
 'cji',
 'tih',
 'bkc',
 'ava',
 'kap',
 'lmy',
 'ibd',
 'yha',
 'mph',
 'plv',
 'ddo',
 'kum',
 'sij',
 'kzi',
 'shj',
 'mbb',
 'rob',
 'tjg',
 'lit',
 'pol',
 'ltz',
 'btw',
 'aba',
 'meu',
 'kmn',
 'kkl',
 'tgl',
 'hat',
 'aym',
 'nio',
 'mtp',
 'yag',
 'gwa',
 'ssf',
 'udi',
 'htu',
 'mzh',
 'aar',
 'doo',
 'apb',
 'vnm',
 'llp',
 'gug',
 'ket',
 'kdt',
 'khv',
 'kij',
 'hin',
 'irh',
 'dds',
 'ido',
 'jow',
 'wri',
 'plj',
 'mad',
 'gbu',
 'erk',
 'tha',
 'dzg',
 'bcl',
 'twu',
 'kzu',
 'adj',
 'pui',
 'rug',
 'pqe',
 'gaz',
 'xmz',
 'bhp',
 'dak',
 'ben',
 'map',
 'azj',
 'ran',
 'khk',
 'lao',
 'dnn',
 'daj',
 'smo',
 'spa',
 'kpt',
 'aqc',
 'mww',
 'bmi',
 'hrv',
 'pau',
 'ihp',
 'vep',
 'sjm',
 'cbr',
 'csb',
 'laf',
 'srn',
 'sei',
 'kjd',
 'ltg',
 'shw',
 'gan',
 'bos',
 'nig',
 'bhq',
 'rus',
 'xkl',
 'hvn',
 'abi',
 'kgo',
 'bnf',
 'waw',
 'hbs',
 '

In [19]:
#Reading ISO639 and Language extracted from SIL Website and Joining to get Language with respect to IS0639
code= pd.read_csv('iso-639-3.tab', encoding='utf8',sep='\t')
mp=code.drop(['Part1','Part2B','Part2T','Scope','Language_Type','Comment'],axis=1)
da=pd.DataFrame(da)
da.columns=['Id']
daa=pd.merge(da,mp,on='Id')
daa

,Id,Ref_Name
0,mkd,Macedonian
1,hdy,Hadiyya
2,lml,Hano
3,alp,Alune
4,ebr,Ebrié
...,...,...
692,uig,Uighur
693,wew,Wejewa
694,xbr,Kambera
695,gor,Gorontalo


Defining Function to mutate data to GraphQL

In [20]:
def test_mutation_result1(client,params):
    query = gql(
        """
          mutation AddISOLanguage($iso: [AddISOLanguageInput!]!) {
          addISOLanguage(input: $iso)  {
            iSOLanguage {
              iso639
              name
            }
  }
}

        """
    )
    result = client.execute(query, variable_values=params)

In [ ]:
#Loading the data into GraphQL by calling function test_mutation_result2
i=0
while i<=len(daa):
        params={"iso":{"iso639":daa.loc[i,"Id"], "name":daa.loc[i,"Ref_Name"]}
           }
       
        test_mutation_result1(client,params)
        i=i+1

Mutating Word to GraphQL

In [21]:
label

Index(['pam_language', 'drl_language', 'gor_language', 'xbr_language',
       'nap_language', 'wew_language', 'uig_language', 'cmn_language',
       'guk_language', 'iba_language',
       ...
       'tkr_language', 'avi_language', 'hbn_language', 'akg_language',
       'fud_language', 'ebr_language', 'alp_language', 'lml_language',
       'hdy_language', 'mkd_language'],
      dtype='object', length=776)

Defining Function to mutate word to GraphQL

In [ ]:
def test_mutation_result2(client,params):
    query = gql(
        """
          mutation Mutation($word: [AddWordInput!]!) {
              addWord(input:$word) {
                    word {
                          text,
                    isoLanguage{
                            iso639
      }
    }
  }
}

        """
    )
    result = client.execute(query, variable_values=params)

In [ ]:
#Mutating the word to GraphQL by calling function test_mutation_result2
for i in ss:
    iio=i.split('_')[0]
    for j in ss[i]:
        if j is None:
            continue
        else:
            try:
                params={"word":{"text":j, "isoLanguage":{"iso639":iio}}}
                test_mutation_result2(client,params)
            except:
                continue
       